# Neural style transfer

Transfert de style d'une image à une autre

A utiliser pour appliquer un style d'une image sur le tableau créé. 


## Introduction

Style transfer consists in generating an image
with the same "content" as a base image, but with the
"style" of a different picture (typically artistic).
This is achieved through the optimization of a loss function
that has 3 components: "style loss", "content loss",
and "total variation loss":

- The total variation loss imposes local spatial continuity between
the pixels of the combination image, giving it visual coherence.
- The style loss is where the deep learning keeps in --that one is defined
using a deep convolutional neural network. Precisely, it consists in a sum of
L2 distances between the Gram matrices of the representations of
the base image and the style reference image, extracted from
different layers of a convnet (trained on ImageNet). The general idea
is to capture color/texture information at different spatial
scales (fairly large scales --defined by the depth of the layer considered).
- The content loss is a L2 distance between the features of the base
image (extracted from a deep layer) and the features of the combination image,
keeping the generated image close enough to the original one.

**Reference:** [A Neural Algorithm of Artistic Style](
  http://arxiv.org/abs/1508.06576)


## Setup


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import vgg19
import matplotlib.pyplot as plt
from skimage.io import imread
from IPython.display import Image, display

result_path = "/content/drive/Shareddrives/PFE artists/Roxane/"


## Image preprocessing / deprocessing utilities


In [3]:
def get_dim(content_image_path):
  # Récupère les dimensions de l'image cible (longueur set et largeur proportionnelle)
  width, height = keras.preprocessing.image.load_img(content_image_path).size
  img_nrows = 400
  img_ncols = int(width * img_nrows / height)
  return img_nrows, img_ncols


def preprocess_image(image_path):
    # Util function to open, resize and format pictures into appropriate tensors
    img = keras.preprocessing.image.load_img(
        image_path, target_size=target_size
    )
    img = keras.preprocessing.image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return tf.convert_to_tensor(img)


def deprocess_image(x):
    # Util function to convert a tensor into a valid image
    x = x.reshape((*target_size, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype("uint8")
    return x



## Compute the style transfer loss

First, we need to define 4 utility functions:

- `gram_matrix` (used to compute the style loss)
- The `style_loss` function, which keeps the generated image close to the local textures
of the style reference image
- The `content_loss` function, which keeps the high-level representation of the
generated image close to that of the base image
- The `total_variation_loss` function, a regularization loss which keeps the generated
image locally-coherent


In [4]:
# The gram matrix of an image tensor (feature-wise outer product)
def gram_matrix(x):
    x = tf.transpose(x, (2, 0, 1))
    features = tf.reshape(x, (tf.shape(x)[0], -1))
    gram = tf.matmul(features, tf.transpose(features))
    return gram


# The "style loss" is designed to maintain
# the style of the reference image in the generated image.
# It is based on the gram matrices (which capture style) of
# feature maps from the style reference image
# and from the generated image
def style_loss(style, result):
    S = gram_matrix(style)
    C = gram_matrix(result)
    channels = 3
    size = target_size[0]*target_size[1]
    return tf.reduce_sum(tf.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))


# An auxiliary loss function
# designed to maintain the "content" of the
# base image in the generated image
def content_loss(content, result):
    return tf.reduce_sum(tf.square(result - content))


# The 3rd loss function, total variation loss,
# designed to keep the generated image locally coherent
def total_variation_loss(x):
  nrows, ncols = target_size
  a = tf.square(
    x[:, : nrows - 1, : ncols - 1, :] - x[:, 1:, : ncols - 1, :]
  )
  b = tf.square(
    x[:, : nrows - 1, : ncols - 1, :] - x[:, : nrows - 1, 1:, :]
  )
  return tf.reduce_sum(tf.pow(a + b, 1.25))



 ## Création du modèle

In [5]:
# Build a VGG19 model loaded with pre-trained ImageNet weights
model = vgg19.VGG19(weights="imagenet", include_top=False)

# Get the symbolic outputs of each "key" layer (we gave them unique names).
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

# Set up a model that returns the activation values for every layer in VGG19 (as a dict).
feature_extractor = keras.Model(inputs=model.inputs, outputs=outputs_dict)


80150528/80134624 [==============================] - 0s 0us/step


Finally, here's the code that computes the style transfer loss.


In [6]:
# List of layers to use for the style loss.
style_layer_names = [
    "block1_conv1",
    "block2_conv1",
    "block3_conv1",
    "block4_conv1",
    "block5_conv1",
]
# The layer to use for the content loss.
content_layer_name = "block5_conv2"


def compute_loss(result_image, content_image, style_image, weights):
    content_weight, style_weight, total_variation_weight = weights
    input_tensor = tf.concat(
        [content_image, style_image, result_image], axis=0
    )
    features = feature_extractor(input_tensor)

    # Initialize the loss
    loss = tf.zeros(shape=())

    # Add content loss
    layer_features = features[content_layer_name]
    content_image_features = layer_features[0, :, :, :]
    combination_features = layer_features[2, :, :, :]
    loss = loss + content_weight * content_loss(
        content_image_features, combination_features
    )
    # Add style loss
    for layer_name in style_layer_names:
        layer_features = features[layer_name]
        style_reference_features = layer_features[1, :, :, :]
        combination_features = layer_features[2, :, :, :]
        sl = style_loss(style_reference_features, combination_features)
        loss += (style_weight / len(style_layer_names)) * sl

    # Add total variation loss
    loss += total_variation_weight * total_variation_loss(result_image)
    return loss

## Add a tf.function decorator to loss & gradient computation

To compile it, and thus make it fast.


In [7]:

@tf.function
def compute_loss_and_grads(result_image, content_image, style_image, weights):
    with tf.GradientTape() as tape:
        loss = compute_loss(result_image, content_image, style_image, weights)
    grads = tape.gradient(loss, result_image)
    return loss, grads



## The training loop

Repeatedly run vanilla gradient descent steps to minimize the loss, and save the
resulting image every 100 iterations.

We decay the learning rate by 0.96 every 100 steps.


In [8]:
def train(content_image, style_image, result_path, result_image, optimizer, weights, epochs, nb_saves=100, verbose=True):
  """ entrainement du réseau sur les images données

  params:
    content_image : image dont le contenu sera utilisé
    style_image : image dont le style sera utilisé
    result_path : chemin où sauvegarder les résultats successifs
    result_image : contient
    nb_saves : nombre de sauvegardes d'images à faire au long de l'entrainement
    verbose : indique s'il faut afficher les détails de l'entrainement
  """

  save_rate = int(epochs/nb_saves) if nb_saves !=0 else epochs
  history = []

  for i in range(1, epochs + 1):
    loss, grads = compute_loss_and_grads(
        result_image, content_image, style_image, weights
    )
    optimizer.apply_gradients([(grads, result_image)])
    # enregistrement de la loss
    history.append(loss)
    if i % save_rate == 0:
      if verbose:
        print("Iteration %d: loss=%.2f" % (i, loss))
      img = deprocess_image(result_image.numpy())
      fname = result_path + "_at_iteration_%d.png" % i
      keras.preprocessing.image.save_img(fname, img)
  # fin de l'entrainement
  print("--- \nFin de l'entrainemnt : loss finale = %.2f"%loss)
  img = deprocess_image(result_image.numpy())
  fname = result_path + "_at_iteration_%d.png" % i
  keras.preprocessing.image.save_img(fname, img)
  return history


In [9]:
# fonction principale : 
def main(content_image_path, style_image_path, result_img_path, epochs=100, saves=10, weights=(2.5e-10, 1e-6, 1e-6), affichage=False):
  # Poids des composants de la loss (à optimiser)
  # total_variation_weight = 1e-6
  # style_weight = 1e-6
  # content_weight = 2.5e-8
  # weights = (content_weight, style_weight, total_variation_weight)

  optimizer = keras.optimizers.SGD(
    keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=100.0, decay_steps=100, decay_rate=0.96
    )
  )

  global target_size
  target_size = get_dim(content_image_path)

  content_image = preprocess_image(content_image_path)
  style_image = preprocess_image(style_image_path)
  result_image = tf.Variable(preprocess_image(content_image_path))

  history = train(
      content_image, 
      style_image, 
      result_img_path, 
      result_image, 
      optimizer, 
      weights, 
      epochs, 
      nb_saves=saves, 
      verbose=affichage
      )
  
  # affichage des résultats
  if affichage:
    figure, axs = plt.subplots(nrows=1,ncols=3,figsize=(15,5) )
    axs[0].imshow(imread(content_image_path))
    plt.title("Image de contenu")
    axs[1].imshow(imread(style_image_path))
    plt.title("Image de style")
    axs[2].imshow(imread(result_img_path + "_at_iteration_%d.png" % epochs))
    plt.title("Résultat")
    plt.show()

    plt.figure()
    plt.title("évolution de la loss le long des epochs")
    plt.plot(history)
    plt.show()

  return history

In [10]:
# premier test avec 2 images
content_image_path = "/content/drive/Shareddrives/PFE artists/data/wikiart/base/Photo/ffcf64f150.jpg"
style_image_path = "/content/drive/Shareddrives/PFE artists/data/wikiart/base/Cubism/victor-brauner_dobrudjan-landscape-1937.jpg"
h = main(content_image_path, style_image_path, result_path+"test2", epochs=50, saves=0, affichage=True)
print(h.numpy())

KeyboardInterrupt: ignored

In [ ]:
style = "/content/drive/Shareddrives/PFE artists/data/wikiart/base/Symbolism/zinaida-serebriakova_mountain-landscape-switzerland-1914.jpg"
content = "/content/drive/Shareddrives/PFE artists/data/wikiart/base/Photo/fffc0836d7.jpg"
# h = main(content, style, result_path+"test3", epochs=200, saves=5, affichage=True)

In [ ]:
style = "/content/drive/Shareddrives/PFE artists/data/wikiart/extended/Pointillism/theo-van-rysselberghe_pines-and-eucalyptus-at-cavelieri-1905.jpg"
content = "/content/drive/Shareddrives/PFE artists/data/wikiart/extended/Impressionism/zinaida-serebriakova_pond-in-tsarskoe-selo-1913.jpg"
# test dans un sens puis dans l'autre
#main(content, style, result_path+"test4", epochs=100, saves=0, affichage=True)
#main(style, content, result_path+"test5", epochs=100, saves=0, affichage=True)

# Etude de l'influence des poids appliqués aux différents losses

objectif : minimiser le temps d'entrainement nécessaire à la convergence
variables :


*   ordre de grandeur des 3 poids
*   rapport content/style
*   rapport content/total



---

comment caractériser la convergence ? 
limite basse sur la pente ? 





In [ ]:
def find_convergence(history, inter=1000):
  # détermine l'époch à partir duquel l'amélioration (diminution) de la loss est considérée comme négligeable
  res = -1
  i=1
  eps = history[0]/inter
  while res<0 and i<len(history):
    test = abs(history[i] - history[i-1])
    if test < eps:
      res = i
    i+=1
  return (res if res>0 else len(history)+1)

def test(ordre_de_grandeur, content_style, content_total, inter=1000):
  style = "/content/drive/Shareddrives/PFE artists/data/wikiart/extended/Pointillism/theo-van-rysselberghe_pines-and-eucalyptus-at-cavelieri-1905.jpg"
  content = "/content/drive/Shareddrives/PFE artists/data/wikiart/extended/Impressionism/zinaida-serebriakova_pond-in-tsarskoe-selo-1913.jpg"

  # teste toutes les combinaisons 
  Lord = len(ordre_de_grandeur)
  Lcs = len(content_style)
  Lct = len(content_total)
  #tab = np.zeros((Lord, Lcs, Lct))

  # ord0 = 1e-6
  cs0 = 2.5e-2
  ct0 = 2.5e-2
  # INITIALISATION
  # total_variation_weight = 1e-6
  # style_weight = 1e-6
  # content_weight = 2.5e-8

  # 1- Optimisation de l'ordre de grandeur
  # ct et cs restent fixés
  tab = np.zeros(Lord)
  for ord in range(Lord):
    w_total = ordre_de_grandeur[ord]
    w_content = w_total * ct0
    w_style = w_content / cs0
    weights = (w_content, w_style, w_total)
    hist = main(content, style, result_path, epochs=100, saves=0, weights=weights)
    tab[ord] = find_convergence(hist, inter=inter)
  print("Optimisation de l'ordre de grandeur\n", tab)
  best_ordi = np.where(tab == tab.min())
  print(best_ordi)
  print(best_ordi[0])
  best_ord = ordre_de_grandeur[int(best_ordi[0][0])]
  print("Ordre de grandeur optimal : ", best_ord)

  # 2- optimisation de content_style
  tab = np.zeros(Lcs)
  for cs in range(Lcs):
    w_total = best_ord
    w_content = w_total * ct0
    w_style = w_content / content_style[cs]
    weights = (w_content, w_style, w_total)
    hist = main(content, style, result_path, epochs=100, saves=0, weights=weights)
    tab[cs] = find_convergence(hist, inter=inter)
  print("Optimisation du rapport content/style \n", tab)
  best_csi = np.where(tab == tab.min())
  print(best_csi)
  print(best_csi[0][0])
  best_cs = ordre_de_grandeur[int(best_csi[0][0])]
  print("Content / style optimal : ", best_cs)

  # 3- optimisation de content_total
  tab = np.zeros(Lct)
  for ct in range(Lct):
    w_total = best_ord
    w_content = w_total * content_total[ct]
    w_style = w_content / best_cs
    weights = (w_content, w_style, w_total)
    hist = main(content, style, result_path, epochs=100, saves=0, weights=weights)
    tab[ct] = find_convergence(hist, inter=inter)
  print("Optimisation du rapport content/ total \n", tab)
  best_cti = np.where(tab == tab.min())
  print(best_cti)
  print(best_cti[0][0])
  best_ct = ordre_de_grandeur[int(best_cti[0][0])]
  print("Content / total optimal : ", best_ct)

  # CONCLUSION
  w_total = best_ord
  w_content = w_total * best_ct
  w_style = w_content / best_cs
  best_weights = (w_content, w_style, w_total)
  return best_weights


In [ ]:
# range de valeurs à tester
ordre_de_grandeur = [1e-5, 1e-6, 1e-7, 1e-8, 1e-9, 1e-10, 1e-11]  #[1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8, 1e-9, 1e-10, 1e-11] 
content_style = [0.01, 0.1, 1, 10, 100]
content_total = [0.01, 0.1, 1, 10, 100]

w = test(ordre_de_grandeur, content_style, content_total, 7000)
#w = test([1e-6], [1e-1, 1e-2], [1e-1, 1e-2])
print(w)

# resultats

+ avec inter=1000 : (1e-14, 1e-9, 1e-9)


# idées :


*   créer d'autres losses pour évaluer la qualité du tableau produit

*   Créer une fonction qui évalue la luminosité / le contraste du tableau pour vérifier qu'il est "visible"

*   Tester d'autres modèles de machine learning ? 


*   Créer une fonction qui évalue si le style d'un tableau est assez marqué pour être utilisé dans un transfert 

